# OPTaaS Integrating with External Code
    
### Note: To run this notebook, you need an API Key. You can get one [here](mailto:charles.brecque@mindfoundry.ai)

Here's how we can generate and run an OPTaaS Task using input from our local ML code or process. This is a simple problem, where we want to maximize the sum of 4 upside-down quadratics, all shifted by a variable amount.

From our local code we will obtain a dictionary of shift amounts, e.g. `param_shifts = {'x1': 0.2, 'x2': 0.5}`. The parameter names, shift values and even the number of parameters may vary.

So we will generate our OPTaaS task on the fly, based on a given value for `param_shifts`.

## Generating OPTaaS parameters

In [1]:
from mindfoundry.optaas.client.parameter import FloatParameter, Distribution

def make_parameters(param_shifts):
    return [FloatParameter(name, minimum=-5, maximum=5, default=0, id=name, distribution=Distribution.UNIFORM)
            for name in param_shifts]

## Generating a scoring function
Since the parameter names will vary, we can iterate over them as a dictionary.

In [2]:
def make_scoring_function(param_shifts): 
    def scoring_function(**params): 
        score = 10
        for param_name, param_value in params.items():
            score = score - (param_value - param_shifts[param_name]) ** 2
        return score
    
    return scoring_function

## Connect to OPTaaS and run your task

In [3]:
from mindfoundry.optaas.client.client import OPTaaSClient
from mindfoundry.optaas.client.goal import Goal

example_param_shifts = {'x1': 0.2, 'x2': 0.5, 'x3': 0.3, 'x4': 0.0}

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

task = client.create_task(
    title='Task',
    parameters=make_parameters(example_param_shifts),
    goal=Goal.max
)

scoring_function = make_scoring_function(example_param_shifts)

best_result = task.run(scoring_function, max_iterations=20)
print("Best Result:", best_result)

Running task "Task" for 20 iterations
(no score threshold set)

Iteration: 0    Score: 9.620000000000001
Configuration: {'x1': 0, 'x2': 0, 'x3': 0, 'x4': 0}

Iteration: 1    Score: 9.620000000000001
Configuration: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0}

Iteration: 2    Score: -17.380000000000003
Configuration: {'x1': -2.5, 'x2': -2.5, 'x3': 2.5, 'x4': 2.5}

Iteration: 3    Score: -13.379999999999999
Configuration: {'x1': 2.5, 'x2': 2.5, 'x3': -2.5, 'x4': -2.5}

Iteration: 4    Score: -14.129999999999999
Configuration: {'x1': -1.25, 'x2': -3.75, 'x3': -1.25, 'x4': 1.25}

Iteration: 5    Score: -29.130000000000003
Configuration: {'x1': 3.75, 'x2': 1.25, 'x3': 3.75, 'x4': -3.75}

Iteration: 6    Score: -11.13
Configuration: {'x1': -3.75, 'x2': -1.25, 'x3': 1.25, 'x4': -1.25}

Iteration: 7    Score: -32.129999999999995
Configuration: {'x1': 1.25, 'x2': 3.75, 'x3': -3.75, 'x4': 3.75}

Iteration: 8    Score: -11.442499999999999
Configuration: {'x1': -1.875, 'x2': 1.875, 'x3': -3.125, '